## Installing the required libraries

In [ ]:
!pip install -q mediapipe-model-maker keras-core

## downloading the dataset from google cloud

In [ ]:
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv

--2023-05-27 17:24:18--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.128, 142.251.10.128, 142.251.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14174600 (14M) [application/octet-stream]
Saving to: ‘data/full_dataset/goemotions_1.csv’

goemotions_1.csv    100%[===================>]  13.52M  8.81MB/s    in 1.5s    

2023-05-27 17:24:20 (8.81 MB/s) - ‘data/full_dataset/goemotions_1.csv’ saved [14174600/14174600]



## Loading the dataset using pandas library

In [ ]:
import pandas as pd
x_train = []
df = pd.read_csv('/content/data/full_dataset/goemotions_1.csv')
df.head()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,...,0,0,0,0,0,0,0,0,0,1


## Data Processing

Removing the unclear sample points

In [ ]:
df = df[df['example_very_unclear'] == False]

Dropping the useless data columns

In [ ]:
df.drop(['id',	'author',	'subreddit'	,'link_id'	,'parent_id'	,'created_utc'	,'rater_id',	'example_very_unclear'],inplace=True,axis=1)
df.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,"You do right, if you don't care then fuck 'em!",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,Right? Considering it’s such an important docu...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
(df.iloc[:, 1:] == 1)

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,False,False,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
69996,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
69997,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
69998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
df['emotion'] = (df.iloc[:, 1:] == 1).idxmax(1)
df.head(5)

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,emotion
0,That game hurt.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,sadness
2,"You do right, if you don't care then fuck 'em!",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,neutral
3,Man I love reddit.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,love
4,"[NAME] was nowhere near them, he was by the Fa...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,neutral
5,Right? Considering it’s such an important docu...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,gratitude


In [ ]:
cols = list(df.columns)
cols.pop(0)
cols

['admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral',
 'emotion']

In [ ]:
df['emotion1'] = df.emotion.apply(cols.index)
df.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,emotion,emotion1
0,That game hurt.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,sadness,25
2,"You do right, if you don't care then fuck 'em!",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,neutral,27
3,Man I love reddit.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,love,18
4,"[NAME] was nowhere near them, he was by the Fa...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,neutral,27
5,Right? Considering it’s such an important docu...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,gratitude,15


In [ ]:
x = df.text.tolist()
y = df.emotion1.tolist()

Creating tensorflow dataset

In [ ]:
import tensorflow as tf
from tensorflow import keras
train_dataset = tf.data.Dataset.from_tensor_slices((x[:1000], y[:1000]))
train_dataset = train_dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
df[['text','emotion']].to_csv('data.csv')

In [ ]:
df[:40000].to_csv('train.csv')
df[40000:45000].to_csv('val.csv')

Using the Mediapipe model maker text classifier

In [ ]:
from mediapipe_model_maker import text_classifier
csv_params = text_classifier.CSVParams(
    text_column='text', label_column='emotion', delimiter=',')
train_data = text_classifier.Dataset.from_csv(
    filename='train.csv',
    csv_params=csv_params)
validation_data = text_classifier.Dataset.from_csv(
    filename='val.csv',
    csv_params=csv_params)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
supported_model = text_classifier.SupportedModels.MOBILEBERT_CLASSIFIER
hparams = text_classifier.HParams(epochs=3, batch_size=32, learning_rate=3e-5, export_dir="bert_exported_models2")
options = text_classifier.TextClassifierOptions(supported_model=supported_model, hparams=hparams)
bert_model = text_classifier.TextClassifier.create(train_data,validation_data, options)

Using existing files at /tmp/model_maker/text_classifier/mobilebert_tiny


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Using existing files at /tmp/model_maker/text_classifier/mobilebert_tiny
Epoch 1/3
1250/1250 [==============================] - 570s 302ms/step - loss: 1834.7748 - test_accuracy: 0.3080 - val_loss: 2.0473 - val_test_accuracy: 0.3928
Epoch 2/3
1250/1250 [==============================] - 354s 283ms/step - loss: 2.0156 - test_accuracy: 0.4090 - val_loss: 1.9945 - val_test_accuracy: 0.4056
Epoch 3/3
1250/1250 [==============================] - 363s 290ms/step - loss: 1.9245 - test_accuracy: 0.4264 - val_loss: 1.9896 - val_test_accuracy: 0.4098


In [ ]:
loss, acc = bert_model.evaluate(validation_data)
print(f'Test loss:{loss}, Test accuracy:{acc}')

157/157 [==============================] - 13s 80ms/step - loss: 1.9896 - test_accuracy: 0.4098
Test loss:1.989622712135315, Test accuracy:0.4097999930381775


In [ ]:
from mediapipe_model_maker import quantization
quantization_config = quantization.QuantizationConfig.for_dynamic()
bert_model.export_model(quantization_config=quantization_config)
bert_model.export_labels(export_dir=options.hparams.export_dir)

In [ ]:
fnet_classifier.save('weights/')

In [ ]:
pred

array([[ 1.0154876 ,  0.46461284,  0.18573295,  0.12951028,  1.2022114 ,
        -0.5405527 , -0.29117277,  0.32608876, -1.766568  , -0.9110432 ,
         0.5638544 , -0.84385926, -1.4943905 , -0.29264846, -0.99796367,
         0.57914716, -2.3186307 , -0.61894995, -1.0248622 , -2.6198466 ,
        -0.45862246, -2.4915853 ,  0.6162668 , -1.9522861 , -1.0838735 ,
        -0.65007126, -1.4498345 ,  2.2543328 ]], dtype=float32)

In [ ]:
!zip -r wgts2.zip weights/

  adding: weights/ (stored 0%)
  adding: weights/variables/ (stored 0%)
  adding: weights/variables/variables.data-00000-of-00001 (deflated 28%)
  adding: weights/variables/variables.index (deflated 78%)
  adding: weights/assets/ (stored 0%)
  adding: weights/fingerprint.pb (stored 0%)
  adding: weights/saved_model.pb (deflated 83%)
  adding: weights/keras_metadata.pb (deflated 68%)
